In [15]:
__author__ = "Miguel Solinas "
__credits__ = "Miguel Solinas "
__version__ = "0.1.0"
__maintainer__ = "Miguel Solinas"
__status__ = "Project"

In [23]:
#importing libraries
import pandas as pd
import numpy as np

In [26]:
#open dataset
file = pd.read_csv("balanced.csv")
dataset = pd.DataFrame(file)

dataset.dtypes
dataset.

type           int64
time         float64
sender         int64
messageID      int64
pos/0        float64
pos/1        float64
pos/2        float64
spd/0        float64
spd/1        float64
class          int64
dtype: object

In [30]:
dataset.class
dataset.classs[dataset.class==1]=0
dataset.classs[dataset.class==5]=1
dataset.to_csv("dataset.csv")
dataset.describe()

SyntaxError: invalid syntax (<ipython-input-30-9c464430adc3>, line 1)

In [14]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="white")

# Generate a large random dataset


# Compute the correlation matrix
corr = dataset.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})




print(" 1) Some of the featues do not deliver major information\n") 
print(" 2) The feature type is usefull, it contains just one value (4)\n")
print(" 3) Pos/2 is not correlated to any feature and it does not change along the samples it is always close to one value\n ")
print(" 4) I can not ensure if there is a correlation between the feature class and the rest ot the features \n")
print(" 5) Dataset not well balanced clas1 = ", np.count_nonzero(dataset.classs==0), "class2 =",np.count_nonzero(dataset.classs==1))

AttributeError: 'dataset_setup' object has no attribute 'corr'

In [11]:
import dataset_setup as ds
import model_nn as model
import tensorflow as tf

tf.reset_default_graph()
features= [ 'type', 'sender', 'messageID', 'pos/2','spd/0', 'spd/1', 'pos/2']
targets= ['class']
dataset = ds.dataset_setup(features,targets)
dataset.input_data("balanced.csv")
new_model = model.model_nn( dataset, 1000, 128,0.001, [len(features),20,2])
new_model.train()



Training samples  (3043, 10) 
Validation samples (320, 10) 
Testing samples (659, 10)


C:\Users\Maturrin\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


num_epochs n° 0 accuracy VALIDATION 0.8378687
num_epochs n° 0 accuracy TESTING 0.8597561
Loss  1302.7018
Loss  22.404552
num_epochs n° 100 accuracy VALIDATION 0.34006843
num_epochs n° 100 accuracy TESTING 0.31134742
Loss  42.848892
Loss  16.13934
num_epochs n° 200 accuracy VALIDATION 0.839987
num_epochs n° 200 accuracy TESTING 0.8597561
Loss  88.59337
Loss  26.007027
num_epochs n° 300 accuracy VALIDATION 0.8166857
num_epochs n° 300 accuracy TESTING 0.8279484
Loss  30.9599
Loss  125.03969
num_epochs n° 400 accuracy VALIDATION 0.84210527
num_epochs n° 400 accuracy TESTING 0.8597561
Loss  35.800903
Loss  17.19749
num_epochs n° 500 accuracy VALIDATION 0.8018576
num_epochs n° 500 accuracy TESTING 0.82685155
Loss  12.303603
Loss  11.63763
num_epochs n° 600 accuracy VALIDATION 0.63027537
num_epochs n° 600 accuracy TESTING 0.6722003
Loss  4.13377
Loss  29.777328
num_epochs n° 700 accuracy VALIDATION 0.84210527
num_epochs n° 700 accuracy TESTING 0.8597561
Loss  131.57169
Loss  10.281744
num_epo